In [1]:
from importlib.metadata import version
import torch

print("torch version:", version("torch"))

torch version: 2.10.0


In [12]:
!echo {0..1000} > numbers.txt

In [3]:
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []
        
        # Modification
        # token_ids = tokenizer.encode(txt, allowed_special={"|<endoftext>|"})
        token_ids = [int(i) for i in txt.strip().split()] # assuming using numbers.txt

        # Use a sliding window to chunk the input text into overlapping sequences of max_length length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i : i + max_length]
            target_chunk = token_ids[i + 1 : i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
            
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, index):
        return self.input_ids[index], self.target_ids[index]

In [4]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True, num_workers=0):
    # Init tokenizer
    # tokenizer = tiktoken.get_encoding("gpt2")
    tokenizer = None
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, num_workers=num_workers)
    return dataloader 

In [13]:
with open("./numbers.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [ ]:
# Test dataloader with batchsize=1 and context size = 4. Basically takes one row at a time.
dl = create_dataloader_v1(raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)
data_iter = iter(dl)
first_batch = next(data_iter)
print(first_batch)
second_batch = next(data_iter)
print(second_batch)
for batch in dl:
    pass
last_batch = batch
print(last_batch)

[tensor([[0, 1, 2, 3]]), tensor([[1, 2, 3, 4]])]
[tensor([[1, 2, 3, 4]]), tensor([[2, 3, 4, 5]])]
[tensor([[996, 997, 998, 999]]), tensor([[ 997,  998,  999, 1000]])]


In [ ]:
# Test with batch_size = 2. Takes two rows at a time, and if you have a gpu, processes them in parallel.
dataloader = create_dataloader_v1(raw_text, batch_size=2, max_length=4, stride=4, shuffle=False)
for inputs, targets in dataloader:
    pass
print("Inputs:\n", inputs)
print("Targets:\n", targets)

Inputs:
 tensor([[992, 993, 994, 995],
        [996, 997, 998, 999]])
Targets:
 tensor([[ 993,  994,  995,  996],
        [ 997,  998,  999, 1000]])


In [20]:
# Now, with shuffling:
torch.manual_seed(123)
dataloader = create_dataloader_v1(raw_text, batch_size=2, max_length=4, stride=4, shuffle=True)
for inputs, targets in dataloader:
    pass
print("Inputs:\n", inputs)
print("Targets:\n", targets)

Inputs:
 tensor([[880, 881, 882, 883],
        [112, 113, 114, 115]])
Targets:
 tensor([[881, 882, 883, 884],
        [113, 114, 115, 116]])
